&nbsp;

# 19 - Théorie des modèles SciML

---

&nbsp;

## 1. Introduction

> #### De quoi parle-t-on quand on parle d'un système *dynamique* ?

Un système sera dit dynamique si sont état évolue selon une règle d'évolution dans le temps (et pas juste des corrélations). La "dynamique" parle de ce qui va arriver, pas seulement de ce qui est observé. Les lois physiques sont causales, locales dans le temps et invariantes, ainsi on cherche pas :

$$
P(X_{t+1} | X_t)
$$

Mais plutôt :

$$
\frac{dX}{dt} = F(X)
$$

Mathématiquement, la caractéristique de *dynamisme* s'attribue à la condition de 4 propriétés clés :

- une variable d'état : $X(t) \in \mathbb{R}^n$ (l'état contient toute les informations nécessaires pour "prédire" le futur)
- une loi d'évolution : $\frac{dX}{dt} = F(X) \space \text{avec} \space F:\mathbb{R}^n \to \mathbb{R}^n$ (où $F$ est la dynamique en question)
- une localité temporelle (causalité) : $\dot{X}(t) = F(X(t))$ (le futur dépend du présent, pas du futur)
- une invariance dans le temps : $F_t = F$ (c'est une loi)

&nbsp;

Dans la mesure où l'on souhaite, plus basiquement, prédire les valeurs futures de SST dans la zone Manche. Il y a deux approches fondamentalement différentes pour prédire :

- une prédiction discrète, on apprend : $X(t + \Delta t) = G(X(t))$ (on apprend une trajectoire, c'est ce que font les régressions classiques, les RNN, les modèles autorégressifs, etc.)
- une prédiction continue, on apprend : $\frac{dX}{dt} = F(X)$

Sans creuser plus dans le détail du raisonnement mathématique, l'approche continue est plus informative, ce qui au final fournit des prédictions plus robustes. Nous nous souhaitons apprendre la loi, pas la trajectoire, car la possibilité d'interprétatiblité que nous offre une telle approche prime sur la simplicité des modèles discrets.

&nbsp;

---

&nbsp;

## 2. Panorama et choix des modèles SciML

Commençons par énoncer que le choix d'un modèle SciML n'est pas indépendant de la représentation d'état. C'est-à-dire que nos 2 états réduits possèdent chacuns des caractéristiques encourageant l'utilisation d'un modèle plus qu'un autre. On ne modèlise pas la même dynamique dans les deux cas.

&nbsp;

> #### Quelles sont les grandes familles de modèles SciML ?

On l'a énoncé plusieurs fois, une approche continue implique le principe suivant :

$$
\frac{dX}{dt} = F(X)
$$

&nbsp;

Chacune des familles se distinguent par la manière dont elles représentent $F$ et par les hypothèses qu'elles imposent.

#### **A. Dynamique symbolique**

Ici, on cherche une équation :

$$
F(X) = \underset{k}{\sum} \xi_k \phi_k(X)
$$

Cela implique une structure explicite, de forme fermée avec une interprétation physique assez directe. On parle des méthodes dites *sparse regression* (SINDy par exemple).

Une famille particulièrement efficace à appliquer à l'état sélectionné, car s'installe une parcimonie physique naturelle et résulte d'équations interprétables (cela reste pertinent de l'appliquer également à l'état réduit obtenu via PCA).

#### **B. Dynamique paramétrique universelle**

Ici, on approxime $F$ sans hypothèse de forme.

$$
F(X)\approx f_\theta(X)
$$

Cela se présente de la forme d'un réseau de neurone ayant plusieurs avantages : flexibilité, peu d'apriori, boîte noire. On parle des méthodes Neural ODE/PDE.

Inversement pour cette famille. Le conditionnement, la légèreté en bruit et la *smoothness* de la dynamique rendue, sont appréciés lors du montage d'un réseau de neurone (également applicable à l'état sélectionné mais tendance au bruitage et à l'hétérogénéité).

#### **C. Dynamique contrainte par la physique**

Ici, on pose l'hypothèse que l'on connaît une partie de l'équation.

$$
F(X) = F_{physique}(X) + F_{ML}(X)
$$

Dès lors on impose la loi physique et on apprend la correction par une forte régularisation. Sont comprises dans cette famille, les méthodes PINNs/Hybrid Models.

#### **D. Dynamique linéarisée en espace étendu**

Dans cette famille, on admet un changement d'espace.

$$
\mathcal{z} = g(X), \space \frac{d\mathcal{z}}{dt} = K\mathcal{z}
$$

Cela s'harmonise correctement avec les dynamiques linéaires pour des observables non physiques. On parle de la famille des méthodes *Koopman*.

&nbsp;

---

&nbsp;

## 3. Prédispositions

> #### Vers quelles modèles SciML se diriger en priorité ?

Après avoir fixé notre choix d'une approche continue, reste encore à décider lequel ou lesquels des nombreux modèles existants implémenter en priorité. Traitons caractéristiques par caractéristiques constatées dans nos diagnostics :

&nbsp;

1. Nous possédons 2 états réduits d'entrée, l'un induit une dynamique latente (le PCA, un espace latent car c'est un esapce mathématique dans lequel "vit" une dynamique réelle valide, mais non directement observable dû à la synthéticité des composants), l'autre une dynamique physique. On cherche donc des modèles capables de prendre en main un espace physique ou un espace latent lisse.

$\to$ exclusion des modèles purements discrets, dépendants de $\Delta t$ ou sans champs de vecteurs explicite (ARIMA, RNN, Transformers temporels, etc.).

&nbsp;

2. Le notebook 10 nous informe que "peu" de modes expliquent une grande part de la variance. Le notebook 14 Bis semblent expliquer que peu de variables suffisent à expliquer la dynamique (LASSO semble mettre à nulle une grande partie des coefficients). Ces indices, additionnés avec la forte redondance spatiale, semblent indiquer que le système "vit" sur une variété à faible dimension, et traduit que peu de degrés de libertés effectifs subsistent en son sein.

$\to$ favorise la parcimonie, les modèles structurés et les bases réduites (affinité pour SINDy et Koopman).

$\to$ structures dominantes favorisant les champs de vecteurs réguliers (affinité pour Neural ODE et Koopman).

&nbsp;

3. Au cours des différentes analyses temporelles (notebooks 6, 7 et 9), les séries temporelles ont toujours été relativement lisses (pas de variations trop rapides et pas de comportements erratiques instantannés). Les dérivées ne sont pas explosives (stabilité numérique dans l'ensemble). De ces faits, une prédiction à court terme ne semble pas absurde, on dira donc que le système n'est pas chaotique à court terme. Le notebook 17 encourage la validation d'une hypothèse de localité concernant la dynamique de l'état.

$\to$ favorise l'apprentissage locale (affinité pour Neural ODE plutôt que RNN).

$\to$ exclusion des modèles qui nécessitent une fenêtre temporelle longue comme LSTM ou tout autre modèle avec une attention temporelle profonde.

&nbsp;

De ces quelques constats brillent déjà quelques noms :

- Neural ODE (Neural SDE par extension)
- SINDy
- Koopman (EDMD par extension, linéarisation spectrale)
- indirectement les modèles hybrides (UDE et *Physics-informed ML*)

Nous nous accorderons ainsi pour une modélisation SciML avec ces premiers candidats.
 
